"""
==============================================================================
04 - TRAIN MODELS FOR ADDITIONAL HORIZONS (3 and 5 bars ahead)
==============================================================================

PURPOSE OF THIS NOTEBOOK:
-------------------------
Train XGBoost and CNN-LSTM models for horizons 3 and 5.
Horizon 1 models are already trained in notebooks 02 and 03.

RUN ORDER:
----------
Each cell can be run independently after the data loading cells.
This allows you to control long-running training step by step.

ESTIMATED TIME:
---------------
- Data loading: ~1 minute
- Each XGBoost: ~5-10 minutes  
- Each CNN-LSTM: ~5-15 minutes
- Total: ~30-60 minutes for all 4 models
"""

# 04 - Train Additional Horizons

This notebook trains models for horizons 3 and 5.
Each training step is a separate cell for easy debugging.

In [2]:
# ==============================================================================
# IMPORTS AND SETUP
# ==============================================================================

import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

import numpy as np
import warnings
warnings.filterwarnings('ignore')

from src.data.loader import load_and_merge_data, split_data_by_time
from src.labeling.oracle import create_oracle_labels
from src.features.builder import prepare_features
from src.features.indicators import get_indicator_columns
from src.models.xgb import XGBBaseline
from src.models.cnn_lstm import CNNLSTMModel

print("✅ Imports loaded")

✅ Imports loaded


In [3]:
# ==============================================================================
# CONFIGURATION
# ==============================================================================

SIGMA = 4
THRESHOLD = 0.0002
TRAIN_END = "2025-06-30"
TEST_START = "2025-07-01"
MODEL_DIR = 'models_artifacts'

# Store all results for final summary
all_results = {}

print("📋 Configuration:")
print(f"   Oracle: sigma={SIGMA}, threshold={THRESHOLD}")
print(f"   Train period: up to {TRAIN_END}")
print(f"   Test period: from {TEST_START}")

📋 Configuration:
   Oracle: sigma=4, threshold=0.0002
   Train period: up to 2025-06-30
   Test period: from 2025-07-01


In [4]:
# ==============================================================================
# LOAD DATA (run once, used by all training cells)
# ==============================================================================

print("=" * 60)
print("📥 LOADING DATA")
print("=" * 60)

df = load_and_merge_data(end_date='2025-12-31')
df = create_oracle_labels(df, sigma=SIGMA, threshold=THRESHOLD)

print(f"✅ Loaded {len(df):,} rows")

# Show label distribution
label_dist = df['target'].value_counts(normalize=True).sort_index()
label_names = {0: 'DOWN', 1: 'SIDEWAYS', 2: 'UP'}
print("\nLabel distribution:")
for label, pct in label_dist.items():
    print(f"   {label_names[label]:8s}: {pct*100:5.1f}%")

📥 LOADING DATA

📥 LOADING DATA

1️⃣ Loading klines...
   Loaded 70,752 rows

2️⃣ Loading funding rates...
   Merged funding rates

3️⃣ Loading volume breakdown...
   Merged volume breakdown
⏰ Filtered until: 2025-12-31

📊 Data Summary:
   Rows: 70,081
   Columns: 16
   Date range: 2024-01-01 00:00:00+00:00 to 2025-12-31 00:00:00+00:00
   Columns: ['time', 'open', 'high', 'low', 'close', 'volume', 'quote_volume', 'taker_buy_volume', 'taker_buy_quote_volume', 'count', 'funding_interval_hours', 'last_funding_rate', 'buy_vol', 'sell_vol', 'total_vol', 'log_return']
✅ Loaded 70,081 rows

Label distribution:
   DOWN    :  29.3%
   SIDEWAYS:  38.4%
   UP      :  32.2%


---
## HORIZON 3 (45 minutes ahead)
---

In [5]:
# ==============================================================================
# PREPARE DATA FOR HORIZON 3
# ==============================================================================

HORIZON = 3
print(f"\n{'='*60}")
print(f"🎯 PREPARING DATA FOR HORIZON={HORIZON} ({HORIZON * 15} minutes ahead)")
print("="*60)

all_results[HORIZON] = {}

# Prepare features with horizon shift
df_features_h3, group_map = prepare_features(df.copy(), horizon=HORIZON)
print(f"   Generated {sum(len(c) for c in group_map.values())} features")

# Split data
train_df_h3, val_df_h3, test_df_h3 = split_data_by_time(
    df_features_h3, train_end=TRAIN_END, test_start=TEST_START, val_ratio=0.1
)
print(f"   Train: {len(train_df_h3):,}, Val: {len(val_df_h3):,}, Test: {len(test_df_h3):,}")

# Get feature columns
feature_cols = get_indicator_columns(
    df_features_h3, exclude_cols=['time', 'target', 'smoothed_close', 'smooth_slope']
)
feature_cols = [c for c in feature_cols if c in train_df_h3.columns]

# Convert to numpy
X_train_h3 = np.nan_to_num(train_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_train_h3 = train_df_h3['target'].values.astype(int)
X_val_h3 = np.nan_to_num(val_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_val_h3 = val_df_h3['target'].values.astype(int)
X_test_h3 = np.nan_to_num(test_df_h3[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_test_h3 = test_df_h3['target'].values.astype(int)

print("✅ Data ready for horizon 3")


🎯 PREPARING DATA FOR HORIZON=3 (45 minutes ahead)

🔧 PREPARING FEATURES
Groups to add: ['momentum', 'overlap', 'trend', 'volatility', 'volume', 'statistics', 'candle']

📊 Adding indicators...
  -> Processing group: momentum
     Added 14 features: MOM_10, TSI_13_25_13, TSIs_13_25_13, STOCHh_14_3_3, WILLR_14...
  -> Processing group: overlap
[!] VWAP requires an ordered DatetimeIndex.
     Added 7 features: HMA_9, SMA_200, EMA_20, EMA_50, EMA_100...
  -> Processing group: trend
     Added 17 features: DMN_14, MACD_12_26_9, ADX_14, CKSPl_10_3_20, CKSPs_10_3_20...
  -> Processing group: volatility
       BBL_20_2.0_2.0  BBM_20_2.0_2.0  BBU_20_2.0_2.0  BBB_20_2.0_2.0  \
0                 NaN             NaN             NaN             NaN   
1                 NaN             NaN             NaN             NaN   
2                 NaN             NaN             NaN             NaN   
3                 NaN             NaN             NaN             NaN   
4                 NaN           

In [6]:
# ==============================================================================
# TRAIN XGBOOST FOR HORIZON 3
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🌲 TRAINING XGBOOST (Horizon={HORIZON})")
print("─"*60)

xgb_h3 = XGBBaseline(n_classes=3, device='cuda', random_state=42)

print("[1/3] Training...")
xgb_h3.fit(X_train_h3, y_train_h3, X_val_h3, y_val_h3, feature_names=feature_cols)

print("[2/3] Tuning hyperparameters...")
best_params = xgb_h3.tune(X_train_h3, y_train_h3, n_iter=15, cv_splits=3, scoring='f1_weighted')

print("\n[3/3] Evaluating...")
xgb_metrics_h3 = xgb_h3.evaluate(X_test_h3, y_test_h3)

print(f"\n✅ XGBoost H=3 Results:")
print(f"   Accuracy: {xgb_metrics_h3['accuracy']:.4f}")
print(f"   F1 Weighted: {xgb_metrics_h3['f1_weighted']:.4f}")

# Save
Path(MODEL_DIR).mkdir(exist_ok=True)
xgb_h3.save(MODEL_DIR, name='xgb_baseline_h3')
all_results[3]['XGBoost'] = xgb_metrics_h3
print(f"💾 Saved: {MODEL_DIR}/xgb_baseline_h3_model.joblib")


────────────────────────────────────────────────────────────
🌲 TRAINING XGBOOST (Horizon=3)
────────────────────────────────────────────────────────────
[1/3] Training...
[2/3] Tuning hyperparameters...

🔧 Tuning hyperparameters with TimeSeriesSplit...
Fitting 3 folds for each of 15 candidates, totalling 45 fits

✅ Best CV score: 0.4952
Best parameters: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}

[3/3] Evaluating...

✅ XGBoost H=3 Results:
   Accuracy: 0.5357
   F1 Weighted: 0.5188
✅ Model saved to models_artifacts
💾 Saved: models_artifacts/xgb_baseline_h3_model.joblib


In [7]:
# ==============================================================================
# TRAIN CNN-LSTM FOR HORIZON 3
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🧠 TRAINING CNN-LSTM (Horizon={HORIZON})")
print("─"*60)

cnn_h3 = CNNLSTMModel(
    n_classes=3,
    lookback=32,
    conv_filters=64,
    lstm_units=64,
    dropout=0.3,
    learning_rate=0.0007,  # Reduced 30% for stability
    device='cuda'
)

print("Training with early stopping (patience=5)...")
history = cnn_h3.fit(
    X_train_h3, y_train_h3,
    X_val_h3, y_val_h3,
    epochs=30,
    batch_size=128,
    patience=5
)

cnn_metrics_h3 = cnn_h3.evaluate(X_test_h3, y_test_h3)

print(f"\n✅ CNN-LSTM H=3 Results:")
print(f"   Accuracy: {cnn_metrics_h3['accuracy']:.4f}")
print(f"   F1 Weighted: {cnn_metrics_h3['f1_weighted']:.4f}")

cnn_h3.save(MODEL_DIR, name='cnn_lstm_h3')
all_results[3]['CNN-LSTM'] = cnn_metrics_h3
print(f"💾 Saved: {MODEL_DIR}/cnn_lstm_h3_model.keras")


────────────────────────────────────────────────────────────
🧠 TRAINING CNN-LSTM (Horizon=3)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0
Training with early stopping (patience=5)...

📊 Preparing data...
📊 Creating sequences (lookback=32)...
   Train sequences: (46915, 32, 62)
   Val sequences: (5184, 32, 62)

🏗️ Building model...


I0000 00:00:1769080464.787728    1491 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21720 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "cnn_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 32, 62)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 64)         │        11,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv1 (Activation)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 32, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv2 (BatchNormalization)   │ (None, 32, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv2 (Activation)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv2 (Dropout)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_lstm1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_lstm1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 190,147 (742.76 KB)

 Trainable params: 189,635 (740.76 KB)

 Non-trainable params: 512 (2.00 KB)


🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.4617 - loss: 1.0368 - val_accuracy: 0.4361 - val_loss: 1.0788 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.5043 - loss: 0.9957 - val_accuracy: 0.4369 - val_loss: 1.1011 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5291 - loss: 0.9632 - val_accuracy: 0.4307 - val_loss: 1.0955 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.5591 - loss: 0.9226 - val_accuracy: 0.3999 - val_loss: 1.1588 - learning_rate: 7.0000e-04
Epoch 5/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.5917 - loss: 0.8740 - val_accuracy: 0.4008 - val_loss: 1.1965 - learning_rate: 7.0000e-04
Epoch 6/30
366/367 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6108 - loss: 0.8391
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0002099999925121665.
36

---
## HORIZON 5 (75 minutes ahead)
---

In [8]:
# ==============================================================================
# PREPARE DATA FOR HORIZON 5
# ==============================================================================

HORIZON = 5
print(f"\n{'='*60}")
print(f"🎯 PREPARING DATA FOR HORIZON={HORIZON} ({HORIZON * 15} minutes ahead)")
print("="*60)

all_results[HORIZON] = {}

# Prepare features with horizon shift
df_features_h5, group_map = prepare_features(df.copy(), horizon=HORIZON)
print(f"   Generated {sum(len(c) for c in group_map.values())} features")

# Split data
train_df_h5, val_df_h5, test_df_h5 = split_data_by_time(
    df_features_h5, train_end=TRAIN_END, test_start=TEST_START, val_ratio=0.1
)
print(f"   Train: {len(train_df_h5):,}, Val: {len(val_df_h5):,}, Test: {len(test_df_h5):,}")

# Get feature columns
feature_cols = get_indicator_columns(
    df_features_h5, exclude_cols=['time', 'target', 'smoothed_close', 'smooth_slope']
)
feature_cols = [c for c in feature_cols if c in train_df_h5.columns]

# Convert to numpy
X_train_h5 = np.nan_to_num(train_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_train_h5 = train_df_h5['target'].values.astype(int)
X_val_h5 = np.nan_to_num(val_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_val_h5 = val_df_h5['target'].values.astype(int)
X_test_h5 = np.nan_to_num(test_df_h5[feature_cols].values, nan=0.0, posinf=0.0, neginf=0.0)
y_test_h5 = test_df_h5['target'].values.astype(int)

print("✅ Data ready for horizon 5")


🎯 PREPARING DATA FOR HORIZON=5 (75 minutes ahead)

🔧 PREPARING FEATURES
Groups to add: ['momentum', 'overlap', 'trend', 'volatility', 'volume', 'statistics', 'candle']

📊 Adding indicators...
  -> Processing group: momentum
     Added 14 features: MOM_10, TSI_13_25_13, TSIs_13_25_13, STOCHh_14_3_3, WILLR_14...
  -> Processing group: overlap
[!] VWAP requires an ordered DatetimeIndex.
     Added 7 features: HMA_9, SMA_200, EMA_20, EMA_50, EMA_100...
  -> Processing group: trend
     Added 17 features: DMN_14, MACD_12_26_9, ADX_14, CKSPl_10_3_20, CKSPs_10_3_20...
  -> Processing group: volatility
       BBL_20_2.0_2.0  BBM_20_2.0_2.0  BBU_20_2.0_2.0  BBB_20_2.0_2.0  \
0                 NaN             NaN             NaN             NaN   
1                 NaN             NaN             NaN             NaN   
2                 NaN             NaN             NaN             NaN   
3                 NaN             NaN             NaN             NaN   
4                 NaN           

In [9]:
# ==============================================================================
# TRAIN XGBOOST FOR HORIZON 5
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🌲 TRAINING XGBOOST (Horizon={HORIZON})")
print("─"*60)

xgb_h5 = XGBBaseline(n_classes=3, device='cuda', random_state=42)

print("[1/3] Training...")
xgb_h5.fit(X_train_h5, y_train_h5, X_val_h5, y_val_h5, feature_names=feature_cols)

print("[2/3] Tuning hyperparameters...")
best_params = xgb_h5.tune(X_train_h5, y_train_h5, n_iter=15, cv_splits=3, scoring='f1_weighted')

print("\n[3/3] Evaluating...")
xgb_metrics_h5 = xgb_h5.evaluate(X_test_h5, y_test_h5)

print(f"\n✅ XGBoost H=5 Results:")
print(f"   Accuracy: {xgb_metrics_h5['accuracy']:.4f}")
print(f"   F1 Weighted: {xgb_metrics_h5['f1_weighted']:.4f}")

xgb_h5.save(MODEL_DIR, name='xgb_baseline_h5')
all_results[5]['XGBoost'] = xgb_metrics_h5
print(f"💾 Saved: {MODEL_DIR}/xgb_baseline_h5_model.joblib")


────────────────────────────────────────────────────────────
🌲 TRAINING XGBOOST (Horizon=5)
────────────────────────────────────────────────────────────
[1/3] Training...
[2/3] Tuning hyperparameters...

🔧 Tuning hyperparameters with TimeSeriesSplit...
Fitting 3 folds for each of 15 candidates, totalling 45 fits

✅ Best CV score: 0.4261
Best parameters: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 100, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}

[3/3] Evaluating...

✅ XGBoost H=5 Results:
   Accuracy: 0.4946
   F1 Weighted: 0.4662
✅ Model saved to models_artifacts
💾 Saved: models_artifacts/xgb_baseline_h5_model.joblib


In [10]:
# ==============================================================================
# TRAIN CNN-LSTM FOR HORIZON 5
# ==============================================================================

print(f"\n{'─'*60}")
print(f"🧠 TRAINING CNN-LSTM (Horizon={HORIZON})")
print("─"*60)

cnn_h5 = CNNLSTMModel(
    n_classes=3,
    lookback=32,
    conv_filters=64,
    lstm_units=64,
    dropout=0.3,
    learning_rate=0.0007,
    device='cuda'
)

print("Training with early stopping (patience=5)...")
history = cnn_h5.fit(
    X_train_h5, y_train_h5,
    X_val_h5, y_val_h5,
    epochs=30,
    batch_size=128,
    patience=5
)

cnn_metrics_h5 = cnn_h5.evaluate(X_test_h5, y_test_h5)

print(f"\n✅ CNN-LSTM H=5 Results:")
print(f"   Accuracy: {cnn_metrics_h5['accuracy']:.4f}")
print(f"   F1 Weighted: {cnn_metrics_h5['f1_weighted']:.4f}")

cnn_h5.save(MODEL_DIR, name='cnn_lstm_h5')
all_results[5]['CNN-LSTM'] = cnn_metrics_h5
print(f"💾 Saved: {MODEL_DIR}/cnn_lstm_h5_model.keras")


────────────────────────────────────────────────────────────
🧠 TRAINING CNN-LSTM (Horizon=5)
────────────────────────────────────────────────────────────
🚀 GPU detected: /physical_device:GPU:0
Training with early stopping (patience=5)...

📊 Preparing data...
📊 Creating sequences (lookback=32)...
   Train sequences: (46915, 32, 62)
   Val sequences: (5184, 32, 62)

🏗️ Building model...


Model: "cnn_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 32, 62)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 32, 64)         │        11,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv1 (Activation)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 32, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_conv2 (BatchNormalization)   │ (None, 32, 128)        │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ relu_conv2 (Activation)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv2 (Dropout)         │ (None, 32, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_lstm1 (BatchNormalization)   │ (None, 32, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_lstm1 (Dropout)         │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 190,147 (742.76 KB)

 Trainable params: 189,635 (740.76 KB)

 Non-trainable params: 512 (2.00 KB)


🏋️ Training for up to 30 epochs (patience=5)...
Epoch 1/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.4409 - loss: 1.0511 - val_accuracy: 0.4047 - val_loss: 1.1100 - learning_rate: 7.0000e-04
Epoch 2/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.4916 - loss: 1.0075 - val_accuracy: 0.3684 - val_loss: 1.1498 - learning_rate: 7.0000e-04
Epoch 3/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.5368 - loss: 0.9537 - val_accuracy: 0.3573 - val_loss: 1.2136 - learning_rate: 7.0000e-04
Epoch 4/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.5805 - loss: 0.8917 - val_accuracy: 0.3576 - val_loss: 1.2750 - learning_rate: 7.0000e-04
Epoch 5/30
367/367 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.6255 - loss: 0.8198 - val_accuracy: 0.3526 - val_loss: 1.3896 - learning_rate: 7.0000e-04
Epoch 6/30
364/367 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6481 - loss: 0.7738
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0002099999925121665.
367

---
## FINAL SUMMARY
---

In [11]:
# ==============================================================================
# SUMMARY
# ==============================================================================

print("\n" + "=" * 60)
print("📋 TRAINING COMPLETE - SUMMARY")
print("=" * 60)

print(f"\n{'Horizon':<10} {'Model':<12} {'Accuracy':<12} {'F1 Weighted':<12}")
print("─" * 48)

for h in [3, 5]:
    if h in all_results:
        for model in ['XGBoost', 'CNN-LSTM']:
            if model in all_results[h]:
                m = all_results[h][model]
                print(f"{h:<10} {model:<12} {m['accuracy']:.4f}       {m['f1_weighted']:.4f}")

print(f"""
{'─' * 48}

📁 Models saved to: {MODEL_DIR}/
   - xgb_baseline_h3_model.joblib
   - xgb_baseline_h5_model.joblib
   - cnn_lstm_h3_model.keras
   - cnn_lstm_h5_model.keras

🔜 NEXT STEP: Run notebook 05_comparison to see full comparison.

✅ ALL DONE!
""")


📋 TRAINING COMPLETE - SUMMARY

Horizon    Model        Accuracy     F1 Weighted 
────────────────────────────────────────────────
3          XGBoost      0.5357       0.5188
3          CNN-LSTM     0.4777       0.4579
5          XGBoost      0.4946       0.4662
5          CNN-LSTM     0.4605       0.4430

────────────────────────────────────────────────

📁 Models saved to: models_artifacts/
   - xgb_baseline_h3_model.joblib
   - xgb_baseline_h5_model.joblib
   - cnn_lstm_h3_model.keras
   - cnn_lstm_h5_model.keras

🔜 NEXT STEP: Run notebook 05_comparison to see full comparison.

✅ ALL DONE!

